# 1. Import libraries and load the dataset

In [54]:
import numpy as np
from numpy import newaxis
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout, Activation
from keras import optimizers

from attention import Attention
from keras.models import Sequential
from keras import optimizers

In [55]:
df = pd.read_csv("./ETH.csv")
df1 = df.reset_index()['Price']
df1.head()

0    1675.16
1    1666.96
2    1650.96
3    1739.60
4    1742.60
Name: Price, dtype: float64

Data preprocessing

In [56]:
# 3. Scaler data
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

# 2. Split the data into training, test and validate  sets

In [57]:
train_size = int(0.6 * len(df1))
test_size = int(0.3 * len(df1))
val_size = len(df1) - train_size - test_size

train_data = df1[:train_size]
test_data = df1[train_size:train_size+test_size]
val_data = df1[train_size+test_size:]

print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)
print("Validate shape:", val_data.shape)

Train shape: (1214, 1)
Test shape: (607, 1)
Validate shape: (203, 1)


In [58]:
train_size = int(0.7 * len(df1))
test_size = int(0.2 * len(df1))
val_size = len(df1) - train_size - test_size

train_data = df1[:train_size]
test_data = df1[train_size:train_size+test_size]
val_data = df1[train_size+test_size:]

print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)
print("Validate shape:", val_data.shape)

Train shape: (1416, 1)
Test shape: (404, 1)
Validate shape: (204, 1)


In [59]:
train_size = int(0.6 * len(df1))
test_size = int(0.2 * len(df1))
val_size = len(df1) - train_size - test_size

train_data = df1[:train_size]
test_data = df1[train_size:train_size+test_size]
val_data = df1[train_size+test_size:]

print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)
print("Validate shape:", val_data.shape)

Train shape: (1214, 1)
Test shape: (404, 1)
Validate shape: (406, 1)


In [60]:
# 5. Hàm Create Dataset
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, X=0,1,2,3-----99   Y=100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [61]:
#6. Reshape into X=t,t+1,t+2..t+99 and Y=t+100

time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_val, y_val = create_dataset(val_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)
print(len(X_test))

303


In [62]:
# 7. Reshape input to be [samples, time steps, features] which is required for LSTM

X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
X_val = X_val.reshape(X_val.shape[0],X_val.shape[1] , 1)

# 3. Perform RNN-Attention Model

## 3.1 Build the RNN Attention Model

In [63]:
model = Sequential([
    SimpleRNN(100, return_sequences=True, input_shape=(time_step, 1)),
    Attention(),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_3 (SimpleRNN)    (None, 100, 100)          10200     
                                                                 
 attention_3 (Attention)     (None, 128)               35600     
                                                                 
 dense_6 (Dense)             (None, 32)                4128      
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 49,961
Trainable params: 49,961
Non-trainable params: 0
_________________________________________________________________


In [64]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

model.fit(X_train,y_train,validation_data=(X_test, y_test),epochs=200,batch_size=64,verbose=1)

Epoch 1/200
18/18 [==============================] - 5s 101ms/step - loss: 0.0364 - accuracy: 8.9847e-04 - val_loss: 8.0396e-04 - val_accuracy: 0.0000e+00
Epoch 2/200
18/18 [==============================] - 1s 54ms/step - loss: 0.0015 - accuracy: 8.9847e-04 - val_loss: 1.7243e-04 - val_accuracy: 0.0000e+00
Epoch 3/200
18/18 [==============================] - 1s 55ms/step - loss: 8.9737e-04 - accuracy: 8.9847e-04 - val_loss: 7.2205e-05 - val_accuracy: 0.0000e+00
Epoch 4/200
18/18 [==============================] - 1s 50ms/step - loss: 7.2109e-04 - accuracy: 8.9847e-04 - val_loss: 4.4955e-05 - val_accuracy: 0.0000e+00
Epoch 5/200
18/18 [==============================] - 1s 52ms/step - loss: 6.5775e-04 - accuracy: 8.9847e-04 - val_loss: 2.4307e-05 - val_accuracy: 0.0000e+00
Epoch 6/200
18/18 [==============================] - 1s 49ms/step - loss: 5.8970e-04 - accuracy: 8.9847e-04 - val_loss: 2.3425e-05 - val_accuracy: 0.0000e+00
Epoch 7/200
18/18 [==============================] - 1s 49m

18/18 [==============================] - 1s 47ms/step - loss: 4.6375e-04 - accuracy: 8.9847e-04 - val_loss: 9.7194e-06 - val_accuracy: 0.0000e+00
Epoch 53/200
18/18 [==============================] - 1s 46ms/step - loss: 4.8709e-04 - accuracy: 8.9847e-04 - val_loss: 1.2617e-05 - val_accuracy: 0.0000e+00
Epoch 54/200
18/18 [==============================] - 1s 51ms/step - loss: 4.7814e-04 - accuracy: 8.9847e-04 - val_loss: 1.0231e-05 - val_accuracy: 0.0000e+00
Epoch 55/200
18/18 [==============================] - 1s 43ms/step - loss: 5.0345e-04 - accuracy: 8.9847e-04 - val_loss: 1.4546e-05 - val_accuracy: 0.0000e+00
Epoch 56/200
18/18 [==============================] - 1s 45ms/step - loss: 4.5503e-04 - accuracy: 8.9847e-04 - val_loss: 1.5436e-05 - val_accuracy: 0.0000e+00
Epoch 57/200
18/18 [==============================] - 1s 43ms/step - loss: 4.3517e-04 - accuracy: 8.9847e-04 - val_loss: 1.5270e-05 - val_accuracy: 0.0000e+00
Epoch 58/200
18/18 [==============================] - 1s 44

18/18 [==============================] - 1s 62ms/step - loss: 4.5067e-04 - accuracy: 8.9847e-04 - val_loss: 1.7479e-05 - val_accuracy: 0.0000e+00
Epoch 104/200
18/18 [==============================] - 1s 57ms/step - loss: 4.3901e-04 - accuracy: 8.9847e-04 - val_loss: 1.5854e-05 - val_accuracy: 0.0000e+00
Epoch 105/200
18/18 [==============================] - 1s 58ms/step - loss: 4.1152e-04 - accuracy: 8.9847e-04 - val_loss: 1.9083e-05 - val_accuracy: 0.0000e+00
Epoch 106/200
18/18 [==============================] - 1s 54ms/step - loss: 4.1079e-04 - accuracy: 8.9847e-04 - val_loss: 1.5134e-05 - val_accuracy: 0.0000e+00
Epoch 107/200
18/18 [==============================] - 1s 55ms/step - loss: 4.0530e-04 - accuracy: 8.9847e-04 - val_loss: 1.9243e-05 - val_accuracy: 0.0000e+00
Epoch 108/200
18/18 [==============================] - 1s 54ms/step - loss: 3.9489e-04 - accuracy: 8.9847e-04 - val_loss: 7.8550e-06 - val_accuracy: 0.0000e+00
Epoch 109/200
18/18 [==============================] -

18/18 [==============================] - 2s 88ms/step - loss: 3.7443e-04 - accuracy: 8.9847e-04 - val_loss: 5.5716e-06 - val_accuracy: 0.0000e+00
Epoch 155/200
18/18 [==============================] - 1s 80ms/step - loss: 3.8762e-04 - accuracy: 8.9847e-04 - val_loss: 2.7197e-05 - val_accuracy: 0.0000e+00
Epoch 156/200
18/18 [==============================] - 2s 88ms/step - loss: 3.4873e-04 - accuracy: 8.9847e-04 - val_loss: 9.8427e-06 - val_accuracy: 0.0000e+00
Epoch 157/200
18/18 [==============================] - 2s 90ms/step - loss: 3.5190e-04 - accuracy: 8.9847e-04 - val_loss: 7.1177e-06 - val_accuracy: 0.0000e+00
Epoch 158/200
18/18 [==============================] - 2s 96ms/step - loss: 3.4767e-04 - accuracy: 8.9847e-04 - val_loss: 1.3288e-05 - val_accuracy: 0.0000e+00
Epoch 159/200
18/18 [==============================] - 2s 89ms/step - loss: 3.5887e-04 - accuracy: 8.9847e-04 - val_loss: 7.4450e-06 - val_accuracy: 0.0000e+00
Epoch 160/200
18/18 [==============================] -

In [65]:
def evaluate_rnn_attention_model(actual, X_data):
    y_pred = model.predict(X_data)
    mse = mean_squared_error(actual, y_pred)
    mae = mean_absolute_error(actual, y_pred)
    mape = mean_absolute_percentage_error(actual, y_pred)
    rmse = mean_squared_error(actual, y_pred, squared=False)
    return y_pred, mse, mae, mape, rmse

## 3.2 Evaluate on Validate data

In [66]:
y_pred_val, val_mse, val_mae, val_mape, val_rmse = evaluate_rnn_attention_model(y_val, X_val)

10/10 [==============================] - 3s 20ms/step


## 3.3 Evaluate on Test data

In [67]:
y_pred_test, test_mse, test_mae, test_mape, test_rmse = evaluate_rnn_attention_model(y_test, X_test)

10/10 [==============================] - 1s 17ms/step


In [68]:
train_predict = model.predict(X_train)
train_predict = scaler.inverse_transform(train_predict)
y_pred_test = scaler.inverse_transform(y_pred_test)
y_pred_val = scaler.inverse_transform(y_pred_val)

35/35 [==============================] - 1s 20ms/step


In [69]:
# 13. Dự báo 30 ngày tiếp theo
x_input=val_data[60:].reshape(1,-1)
x_input.shape

temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=142
i=0
while(i<30):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input= x_input.reshape(1,-1)
        print("x_input:",x_input)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps, 1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1


0 day input [0.02738874 0.02700352 0.02731677 0.02857191 0.02887247 0.02685112
 0.027103   0.0246414  0.02515573 0.02420538 0.02429427 0.02404028
 0.02396832 0.0256489  0.02551555 0.02556211 0.02546263 0.02567641
 0.02579494 0.02602565 0.02615264 0.02629446 0.02564466 0.02598543
 0.02744165 0.02778031 0.02761098 0.02397467 0.02485094 0.02389424
 0.0226645  0.03015936 0.03057209 0.03090863 0.03011914 0.0299223
 0.03059114 0.02936987 0.02893385 0.030062   0.03131714 0.03153514
 0.0313383  0.02920054 0.03076259 0.02763003 0.02862271 0.0305213
 0.03404754 0.03323265 0.03473544 0.02976144 0.02665428 0.026504
 0.02379687 0.02886824 0.02923652 0.02670508 0.02692097 0.02099027
 0.02143264 0.0239662  0.02370374 0.02391329 0.0278946  0.03085783
 0.03068216 0.04261975 0.04341347 0.04471095 0.04476598 0.04193609
 0.04236576 0.04330764 0.04483159 0.04291607 0.04035288 0.04101961
 0.04179851 0.04070423 0.03937501 0.04171597 0.03963747 0.04561685
 0.04446542 0.04933571 0.04295629 0.04176253 0.0411423

ValueError: cannot reshape array of size 345 into shape (1,142,1)

In [70]:
metrics = {
    'MSE': [val_mse, test_mse],
    'MAE': [val_mae, test_mae],
    'MAPE': [val_mape, test_mape],
    'RMSE': [val_rmse, test_rmse],
}

acc = pd.DataFrame(metrics, index=['Validate', 'Test'])

print(acc)

               MSE       MAE      MAPE      RMSE
Validate  0.000125  0.007988  0.084599  0.011189
Test      0.000030  0.005003  0.302363  0.005456


In [ ]:
# 14.Vẽ hình

train_data_index = pd.RangeIndex(start=0, stop=train_size, step=1)
plt.plot(scaler.inverse_transform(train_data))

test_data_index = pd.RangeIndex(start=train_size, stop=train_size+test_size, step=1)
plt.plot(test_data_index,scaler.inverse_transform(test_data))
test_data_index = pd.RangeIndex(start=train_size+101, stop=train_size+test_size, step=1)
plt.plot(test_data_index,(y_pred_test))

val_data_index = pd.RangeIndex(start=train_size+test_size, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,scaler.inverse_transform(val_data))
val_data_index = pd.RangeIndex(start=train_size+test_size+101, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,y_pred_val)

prediect_data_index = pd.RangeIndex(start=len(df1)-1, stop=len(df1)+29, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output))

plt.legend(['Train','Test','Predict','Validate','ValidatePred','Predict30days'])
plt.show()